In [30]:
#Note: Part A for the first feature engineering

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
import numpy as np
import torch
from torch import nn, optim, autograd
from math import pi
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
import time
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from torch import nn, optim



torch.manual_seed(123456)
np.random.seed(123456)
Class_crit=0 
num_labels=51 
PFP_PSP_active=1
num_el= 7

A1=np.loadtxt('Labels_T_extra.txt')
A2=np.loadtxt('Labels_extra.txt')
A=np.append(A1,A2,axis=0)
A[:,[0]]=np.reshape(np.arange(0,np.shape(A)[0]),(np.shape(A)[0],1))
A=np.append(A,A[:,[num_el+1]],axis=1) 
B1=np.loadtxt('Phases_T.txt')
B2=np.loadtxt('Phases.txt')
B=np.append(B1,B2,axis=0)
B[:,[0]]=np.reshape(np.arange(0,np.shape(B)[0]),(np.shape(B)[0],1))

C=np.zeros([1,np.shape(B)[1]])
for i in range(np.shape(B)[0]):
    if np.remainder(i,50000)==0:
        print('i=',i)
    if B[i][np.shape(B)[1]-8]!=99999:
        C=np.append(C,B[[i],:],axis=0)
B=np.delete(C,0,axis=0)
C=np.zeros([np.shape(B)[0],np.shape(B)[1]+num_labels]) 
      
IDtoT=np.shape(B)[1]-8            
for i in range(np.shape(B)[0]): 
    C[[i],0:IDtoT]=B[[i],0:IDtoT] 
    C[[i],IDtoT:IDtoT+num_labels]=A[[int(B[i][0])],IDtoT:IDtoT+num_labels] 
    C[[i],IDtoT+num_labels:IDtoT+num_labels+8]=B[[i],IDtoT:IDtoT+8]

C=C[:,1:] 
unique_rows, unique_indices = np.unique(C, axis=0, return_index=True)
unique_matrix = C[np.sort(unique_indices)]
C_unique=np.zeros([np.shape(unique_matrix)[0],1])
for i in range(np.shape(unique_matrix)[0]):
    C_unique[i][0]=i
C_unique=np.append(C_unique,unique_matrix,axis=1)
C=C_unique
D=np.copy(C)
norm=np.zeros([7,51,2])
C1=np.copy(C)

In [29]:
#Note: Part B for the first feature engineering
#To complete the first figure engineering, we  need to execute this section multiple times until we stop getting the message
#"Number of deleted RMPEAs="

import matplotlib.pyplot as plt
import numpy as np
for i in range(IDtoT,IDtoT+num_labels):
    n=[]
    if i>=IDtoT+10 and i<=IDtoT+10+5 and PFP_PSP_active==1:
        pass
    else:
        mn1=C[np.argmin(C[:,i])][i]
        mx1=C[np.argmax(C[:,i])][i]
        norm[0,i-IDtoT,0]=mn1 
        norm[0,i-IDtoT,1]=mx1 
        if mn1!=0 or mx1!=0:
            for j2 in range(np.shape(C)[0]):
                C[j2][i]=1*((C[j2][i]-mn1)/(mx1-mn1))-0
            
    bin_range = (0, 1)
    bin_size = 0.1
    num_bins = int((bin_range[1] - bin_range[0]) / bin_size)
    bins = np.arange(bin_range[0], bin_range[1] + bin_size, bin_size)
    
    
    Z=C[:,[i]]
    hist=plt.hist(Z,bins=bins) 
    plt.yscale('log')
    for j in range(np.shape(hist[0])[0]):
        if hist[0][j]<=20: 
            a1=hist[1][j]
            a2=hist[1][j+1]
            for i1 in range(np.shape(C)[0]):
                if C[i1][i]>=a1 and C[i1][i]<=a2:
                    n=np.append(n,i1)
    if np.shape(n)[0] !=0:
        print('Number of deleted RMPEAs=',np.shape(n)[0])
        C1=np.delete(C1,n.astype(int),axis=0)
C=np.copy(C1)

In [18]:
#Note: Part C for the first feature engineering
#Following the genreation of C from last step, this code generates the final dataset from the first feature engineering
xlo=C[:,0:IDtoT+num_labels] 
ylo=C[:,IDtoT+num_labels:IDtoT+num_labels+8]
num_rows=np.shape(xlo)[0]
permutation = np.random.permutation(num_rows)
xlo = xlo[permutation]
ylo = ylo[permutation]
xlo_test=xlo[0:34458,:]
ylo_test=ylo[0:34458,:]
xlo=np.delete(xlo,range(0,34458),axis=0)
ylo=np.delete(ylo,range(0,34458),axis=0)
comp=xlo[:,1:8]
comp_test=xlo_test[:,1:8]
xlo1=xlo[:,8+2:]
xlo_test1=xlo_test[:,8+2:]

with open('xlo.npy', 'wb') as f:
    np.save(f,xlo1)
with open('ylo.npy', 'wb') as f:
    np.save(f,ylo)
with open('xlo_test.npy', 'wb') as f:
    np.save(f,xlo_test1)
with open('ylo_test.npy', 'wb') as f:
    np.save(f,ylo_test)

In [2]:
#Note: Part A of the second feature engineering 
#This code shows which features needs to be removed based on Pearson's correlation analysis

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

A1=np.load('xlo.npy')
A2=np.load('xlo_test.npy')
A=np.append(A1,A2,axis=0)
input_correlation_matrix = np.corrcoef(A.T)
colors = ['blue', 'red']
cmap = ListedColormap(colors)
plt.figure(figsize=(10, 8))
sns.heatmap(np.abs(input_correlation_matrix),cmap=cmap,linewidths=1, linecolor='black', vmin=0.7, center=0.9,vmax=1)
plt.xticks(ticks=[0,5,10,15,20,25,30,35,40,45], labels=[0,5,10,15,20,25,30,35,40,45], fontsize=15)
plt.yticks(ticks=[0,5,10,15,20,25,30,35,40,45], labels=[0,5,10,15,20,25,30,35,40,45],fontsize=15)
plt.tight_layout()
plt.savefig('figure.png', dpi=600)
plt.show()

n=[]
m=[]
for i in range(np.shape(input_correlation_matrix)[0]):
    for j in range(i+1,np.shape(input_correlation_matrix)[0]):
        if input_correlation_matrix[i,j]>0.9 and np.shape(np.where(n==i))[1]==0 and np.shape(np.where(n==j))[1]==0:
            n=np.append(n,j)
            print('(i,j)=',i,j) #note that the second element (i.e., j) should be removed

In [ ]:
#Note: Part B of the second feature engineering 
#Following thie code, the final dataset for starting the training process (i.e. xlo_1.npy,ylo.npy, xlo_test1.npy
#, ylo_test.npy) are generated

A=np.load('xlo.npy')
B=np.load('xlo_test.npy')
n=[7,9,38,40,45,48,49,39,25,26,29,47,34,35,41,46]  #The list of the removed elements (i.e., j from the last part)
A=np.delete(A,n,axis=1)
B=np.delete(B,n,axis=1)

with open('xlo_1.npy','wb') as file:
    np.save(file,A)
with open('xlo_test_1.npy','wb') as file:
    np.save(file,B)